# Day 6 - Self-Paced Practice: Building a Docker Compose Agent

**Objective:** Reinforce the concepts of tool-using agents by building a simple agent that can interact with local Docker Compose files.

**Estimated Time:** 45 minutes

**Introduction:**
A powerful use case for AI agents is interacting with local development environments. In this lab, you will build a simplified Docker Compose agent. It will have tools to read files from your local disk and use its reasoning ability to answer questions about your Docker setup. This is a practical example of how agents can become useful developer assistants.

## 1. Setup

We will set up our environment and create a sample `docker-compose.yml` file for our agent to interact with. This file will be created in the root of the project directory.

In [ ]:
import sys
import os

# Add the project's root directory to the Python path
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from langchain_core.tools import tool
from typing import List
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from utils import setup_llm_client, save_artifact

client, model_name, api_provider = setup_llm_client(model_name="gpt-4o")
llm = ChatOpenAI(model=model_name)

# Create a sample docker-compose file in the project root
docker_compose_content = """
version: '3.8'
services:
  web:
    image: 'nginx:latest'
    ports:
      - '8080:80'
  database:
    image: 'postgres:13'
    environment:
      POSTGRES_PASSWORD: mysecretpassword
"""
save_artifact(docker_compose_content, "docker-compose.yml")

### ⭐ Deeper Dive: The Local Environment as a Tool

This lab demonstrates a profound concept in agentic AI: giving an agent the ability to interact with its **local environment**. While web search is powerful, agents that can read, write, and execute files on your machine are true developer assistants.

Consider the possibilities:
- An agent that can read your entire codebase to answer questions about how different modules interact.
- An agent that can write new files (like boilerplate code or test cases) directly into your project.
- An agent that can execute shell commands to run your test suite or start a development server.

The tools you are building in this lab (`list_compose_files` and `read_compose_file`) are the first simple steps toward this advanced capability. The key is to create well-defined, safe functions that provide a secure bridge between the LLM's reasoning and your file system.

## 2. Your Task

Build a LangChain agent that can list and read Docker Compose files in the project directory.

#### 💡 Pro-Tip: Precision in Tool Docstrings

The agent's ability to choose the correct tool depends almost entirely on the tool's name and its docstring. Be precise and descriptive.

- **Good:** `"Lists all Docker Compose YAML files in the current directory."` This is clear and specific.
- **Less Good:** `"Lists files."` This is too vague. The agent doesn't know *what kind* of files or *where* to look.

For tools that take arguments, like `read_compose_file(filename: str)`, the type hint (`: str`) and argument name (`filename`) are also crucial pieces of information that the agent uses to understand how to call the tool correctly.

In [ ]:
# --- Task 1: Define the Tools --- 

@tool
def list_compose_files() -> List[str]:
    """Lists all Docker Compose YAML files in the project's root directory."""
    try:
        files = os.listdir(project_root)
        return [f for f in files if f.endswith(('.yml', '.yaml')) and 'docker-compose' in f]
    except Exception as e:
        return [f"Error listing files: {e}"]

@tool
def read_compose_file(filename: str) -> str:
    """Reads the content of a specified Docker Compose file from the project's root directory."""
    try:
        filepath = os.path.join(project_root, filename)
        with open(filepath, 'r') as f:
            return f.read()
    except FileNotFoundError:
        return f"Error: File '{filename}' not found."
    except Exception as e:
        return f"Error reading file: {e}"

# --- Task 2: Create the Agent --- 

docker_tools = [list_compose_files, read_compose_file]

docker_agent_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an expert assistant for Docker Compose. You can list and read compose files in the user's project directory."),
    ("user", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

docker_agent = create_tool_calling_agent(llm, docker_tools, docker_agent_prompt)
docker_agent_executor = AgentExecutor(agent=docker_agent, tools=docker_tools, verbose=True)

# --- Task 3: Test the Agent --- 

print("--- Testing agent: Listing files ---")
list_result = docker_agent_executor.invoke({"input": "What docker-compose files are in this directory?"})
print(f"\nFinal Answer: {list_result['output']}")

print("\n--- Testing agent: Reading a file to answer a question ---")
read_result = docker_agent_executor.invoke({"input": "What services are defined in docker-compose.yml? And what ports are exposed for the web service?"})
print(f"\nFinal Answer: {read_result['output']}")

## Lab Conclusion

Excellent! You have built a practical developer assistant that can interact with your local file system. This lab demonstrates how easily you can extend an agent's capabilities by giving it custom tools, allowing it to perform tasks far beyond simple text generation. You could imagine extending this agent with tools to run `docker-compose up` or `docker-compose down`, turning it into a fully functional DevOps assistant.